In [16]:
import requests
from bs4 import BeautifulSoup

In [17]:
mimic_dict = {"hosp": {}, 'icu': {}}

In [18]:
url = 'https://mimic.mit.edu/docs/iv/modules/hosp/'
soup = BeautifulSoup(requests.get(url).text, "html.parser")
hosp_table_name = [i.find('a')['href'] for i in soup.find_all('div', attrs={'class' : 'entry'})]

for i in range(len(hosp_table_name)):
    table_url = f"https://mimic.mit.edu//{hosp_table_name[i]}"
    soup = BeautifulSoup(requests.get(table_url).text, "html.parser")

    all_info = soup.find('div', attrs={'class':'td-content'})
    table_name_header = all_info.find_next() # 테이블 이름

    if table_name_header:
        table_name = table_name_header.get_text().replace(' table','').strip()

        mimic_dict['hosp'][table_name] = {"table_instruction" : "",
                                            "columns_instruction" : {}}
        
        for i in table_name_header.find_next_siblings():
            if i.name == 'h2' and i.get('id') == 'table-columns':
                break
            mimic_dict['hosp'][table_name]['table_instruction'] += i.get_text().replace('\n','').strip() + ' '

    table_columns_header = soup.find('h2', id='table-columns')
    if table_columns_header:
        # 다음 테이블 찾기
        columns_table = table_columns_header.find_next('table')
        if columns_table:
            rows = columns_table.find('tbody').find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    column_name = cols[0].get_text().strip()
                    data_type = cols[1].get_text().strip()
                    mimic_dict['hosp'][table_name]['columns_instruction'][column_name] = {'data_type': data_type}

    detailed_desc_header = soup.find('table')
    if detailed_desc_header:
        x = detailed_desc_header.find_next_siblings()

    for tag in x:
        if tag.name in ['h2', 'h3']:
            col_name = tag.get_text(strip=True).split(',')

            for c in col_name:
                for sib in tag.find_next_siblings():
                    if sib.name in ['h2', 'h3']:
                        break
                    if sib.name == 'p':
                        if c in mimic_dict['hosp'][table_name]['columns_instruction'].keys():
                            mimic_dict['hosp'][table_name]['columns_instruction'][c]['column_description'] = sib.get_text(strip=True)

In [19]:
url = 'https://mimic.mit.edu/docs/iv/modules/icu/'
soup = BeautifulSoup(requests.get(url).text, "html.parser")
icu_table_name = [i.find('a')['href'] for i in soup.find_all('div', attrs={'class' : 'entry'})]

for i in range(len(icu_table_name)):

    table_url = f"https://mimic.mit.edu//{icu_table_name[i]}"
    soup = BeautifulSoup(requests.get(table_url).text, "html.parser")

    all_info = soup.find('div', attrs={'class':'td-content'})
    table_name_header = all_info.find_next() # 테이블 이름

    if table_name_header:
        table_name = table_name_header.get_text().replace(' table','').strip()

        mimic_dict['icu'][table_name] = {"table_instruction" : "",
                                            "columns_instruction" : {}}
        
        for i in table_name_header.find_next_siblings():
            if i.name == 'h1' and i.get('id') == 'table-columns':
                break
            mimic_dict['icu'][table_name]['table_instruction'] += i.get_text().replace('\n','').strip() + ' '


    table_columns_header = soup.find('h1', id='table-columns')
    if table_columns_header:
        # 다음 테이블 찾기
        columns_table = table_columns_header.find_next('table')
        if columns_table:
            rows = columns_table.find('tbody').find_all('tr')
            for row in rows:
                cols = row.find_all('td')
                if len(cols) >= 2:
                    column_name = cols[0].get_text().strip()
                    data_type = cols[1].get_text().strip()
                    mimic_dict['icu'][table_name]['columns_instruction'][column_name] = {'data_type': data_type}

    detailed_desc_header = soup.find('table')
    if detailed_desc_header:
        x = detailed_desc_header.find_next_siblings()

    for tag in x:
        if tag.name in ['h2', 'h3']:
            col_name = tag.get_text(strip=True).split(',')

            for c in col_name:
                for sib in tag.find_next_siblings():
                    if sib.name in ['h2', 'h3']:
                        break
                    if sib.name == 'p':
                        c = c.lower()
                        if c in mimic_dict['icu'][table_name]['columns_instruction'].keys():
                            mimic_dict['icu'][table_name]['columns_instruction'][c]['column_description'] = sib.get_text(strip=True)

In [20]:
import json
with open("mimic_table_info.json", "w", encoding="utf-8") as f:
    json.dump(mimic_dict, f, ensure_ascii=False, indent=2)